In [ ]:
one_mb = 1048576 - 1

def gen_10mb_files(count=0):
    for i in range(count):
        f = open(f"upload/10mb_file{i}","wb")
        f.seek(one_mb*10)
        f.write(b"\0")
        f.close()
        
gen_10mb_files(10000)

In [ ]:
one_gb = 1073741824-1

def gen_1gb_files(count=0):
    for i in range(count):
        f = open(f"upload/1gb_file{i}","wb")
        f.seek(one_gb*1)
        f.write(b"\0")
        f.close()

gen_1gb_files(100)

Reference:
https://blog.taiker.space/python-python-boto3-and-aws-s3/
https://skonik.me/uploading-large-file-to-s3-using-aiobotocore/
https://www.peterbe.com/plog/fastest-way-to-download-a-file-from-s3
https://bryson3gps.wordpress.com/2021/04/01/a-quick-look-at-s3-read-speeds-and-python-lambda-functions/
https://dashbird.io/blog/aws-s3-python-tricks/

https://community.tibco.com/wiki/list-and-download-items-aws-s3-buckets-tibco-spotfirer

In [ ]:
target_bucket = "test-s3-download-performance"
target_file_prefix = "upload/10mb_file"
count = 100

## Use aioboto3 to upload 

In [ ]:
from pprint import pprint
import time
import boto3
import asyncio
import aioboto3

boto3.setup_default_session(profile_name='dev')

async def upload_to_aws(s3, file_path, bucket):
    
    
    try:
        await s3.upload_file(file_path, bucket, file_path)
        return "upload successful"
    except Exception as e:
        return "unable upload to s3" + str(e)

async def async_upload_files(file_prefix, bucket, count=0):
    events = list()
    session = aioboto3.Session()
    async with session.client("s3") as s3:
        for i in range(count):
            events.append(upload_to_aws(s3, f"{file_prefix}{i}", bucket))
        res = await asyncio.gather(*events)
    pprint(res)


s = time.perf_counter()
await async_upload_files(target_file_prefix, target_bucket, count)
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")


# Use aiobotocore to upload

In [ ]:
from pprint import pprint
import time
import boto3
import asyncio
from aiobotocore.session import get_session

boto3.setup_default_session(profile_name='dev')

async def upload_to_aws(s3, file_path, bucket):
    try:
        with open(file_path, "rb") as f:
            await s3.put_object(Bucket = bucket, Key = file_path, Body = f)
            return "upload successful"
    except Exception as e:
        return "unable upload to s3" + str(e)

async def async_upload_files(file_prefix, bucket, count=0):
    events = list()
    session = get_session()
    async with session.create_client("s3") as s3:
        for i in range(count):
            events.append(upload_to_aws(s3, f"{file_prefix}{i}", bucket))
        res = await asyncio.gather(*events)
    pprint(res)


s = time.perf_counter()
await async_upload_files(target_file_prefix, target_bucket, count)
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")


# Use general boto3 for upload

In [ ]:
import time
import boto3
boto3.setup_default_session(profile_name='dev')


def upload_file(s3, file_name, bucket):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    try:
        response = s3.upload_file(file_name, bucket, file_name)
    except ClientError as e:
        print(e)
        return False
    return True

def upload_files(file_prefix, bucket, count=0):
    s3 = boto3.client('s3')
    for i in range(count):
        upload_file(s3, f"{file_prefix}{i}", bucket)
            
s = time.perf_counter()
upload_files(target_file_prefix, target_bucket, count)
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")